Chainlit used to create chat interface
Link: https://docs.chainlit.io/integrations/langchain

Useful Resources:
* Memory: https://www.youtube.com/watch?v=X550Zbz_ROE


In [ ]:
#!pip install huggingface_hub
#!pip install transformers
#!pip install langchain
#!pip install chainlit
#!pip install openai

In [1]:
#import openai
#import chainlit as cl
import os
from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from langchain.chains import ConversationChain
from dotenv import load_dotenv

load_dotenv('.env')

True

In [2]:
# model_id = "gpt2-medium" # 355M parameter
# model_id = "google/gemma-2b-it"
# model_id = "google/gemma-7b"
model_id = "google/flan-ul2"
temperature = 0.1

conv_model = HuggingFaceHub(huggingfacehub_api_token=os.environ['HUGGINGFACEHUB_API_TOKEN'],
                            repo_id=model_id,
                            model_kwargs={"temperature": temperature, "max_new_tokens":200}) # Temperature is randomness of model

c:\Users\Julián Ríos\Desktop\Repository\Machine Learning\Thesis Model\.venv_thesis\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [3]:
from langchain.chains.conversation.memory import ConversationBufferMemory, ConversationSummaryMemory, ConversationSummaryBufferMemory, ConversationKGMemory, ConversationEntityMemory 
memory = ConversationBufferMemory()
conv_chain = ConversationChain(llm=conv_model,
                               verbose=True,
                               memory=memory)

In [4]:
conv_chain.predict(input="Hi, I'm Julian. My TV is broken but is under warranty")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I'm Julian. My TV is broken but is under warranty
AI:

> Finished chain.


"Hi, I'm Julian. What is the brand of your TV?"

In [5]:
conv_chain.predict(input="The screen is broken")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, I'm Julian. My TV is broken but is under warranty
AI: Hi, I'm Julian. What is the brand of your TV?
Human: The screen is broken
AI:

> Finished chain.


'What is the model number of your TV?'

In [6]:
conv_chain.predict(input="Yes, the warranty numbers is A1241")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, I'm Julian. My TV is broken but is under warranty
AI: Hi, I'm Julian. What is the brand of your TV?
Human: The screen is broken
AI: What is the model number of your TV?
Human: Yes, the warranty numbers is A1241
AI:

> Finished chain.


'What is the serial number of your TV?'

In [7]:
# Useful commands
from pprint import pprint
pprint(conv_chain.memory.buffer)

("Human: Hi, I'm Julian. My TV is broken but is under warranty\n"
 "AI: Hi, I'm Julian. What is the brand of your TV?\n"
 'Human: The screen is broken\n'
 'AI: What is the model number of your TV?\n'
 'Human: Yes, the warranty numbers is A1241\n'
 'AI: What is the serial number of your TV?')


In [8]:
# Tokens
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-ul2")

tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

c:\Users\Julián Ríos\Desktop\Repository\Machine Learning\Thesis Model\.venv_thesis\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Julián Ríos\.cache\huggingface\hub\models--google--flan-ul2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.43M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [9]:
test_input = "Good morning everyone"
# tokenizer([test_input])
tokenizer.tokenize(test_input) # to see how the tokenizer works

['▁Good', '▁morning', '▁everyone']

In [10]:
memory.buffer

"Human: Hi, I'm Julian. My TV is broken but is under warranty\nAI: Hi, I'm Julian. What is the brand of your TV?\nHuman: The screen is broken\nAI: What is the model number of your TV?\nHuman: Yes, the warranty numbers is A1241\nAI: What is the serial number of your TV?"

In [11]:
formatted_prompt = conv_chain.prompt.format(input='next input', history=memory.buffer)
formatted_prompt

"The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\nHuman: Hi, I'm Julian. My TV is broken but is under warranty\nAI: Hi, I'm Julian. What is the brand of your TV?\nHuman: The screen is broken\nAI: What is the model number of your TV?\nHuman: Yes, the warranty numbers is A1241\nAI: What is the serial number of your TV?\nHuman: next input\nAI:"

In [12]:
def chat_to_llm(chat_llm):
    conversation_total_tokens = 0
    new_conversation = ConversationChain(llm=chat_llm, 
                                     verbose=False, 
                                     memory=ConversationBufferMemory()
                                     )
    
    while True:
        message = input("Human: ")
        if message=='exit':
            print(f"{conversation_total_tokens} tokens used in total in this conversation")
            break
        if message:
            formatted_prompt = new_conversation.prompt.format(input=message,history=new_conversation.memory.buffer)
            num_tokens = len(tokenizer.tokenize(formatted_prompt))
            conversation_total_tokens += num_tokens
            print(f'tokens sent {num_tokens}')
            response = new_conversation.predict(input=message)
            response_num_tokens = len(tokenizer.tokenize(response))
            conversation_total_tokens += response_num_tokens
            print(f"LLM: {response}")

In [13]:
chat_to_llm(conv_model)

tokens sent 65
LLM: Hi, I'm a chatbot. How can I help you?
tokens sent 89
LLM: Is it the screen or the sound?
tokens sent 105
LLM: It is Samsung.
288 tokens used in total in this conversation
